# Sarcasm Detection Using BERT 

## Task 1: Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import re
import tensorflow_hub as hub
import tensorflow_text
from official.nlp import optimization 
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
from keras.layers import Dense
from tensorflow.keras.models import model_from_json
import math
from sklearn.metrics import classification_report,confusion_matrix
import os

tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-08-05 03:52:11.210398: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-05 03:52:12.242326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-08-05 03:52:12.242390: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-08-05 03:52:15.173095: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

## Task 2: Load Dataset

In [2]:
train = pd.read_csv('./usercode/train.csv')
test = pd.read_csv('./usercode/test.csv')
val = pd.read_csv('./usercode/validate.csv')
X_train = train[["text"]].copy()
X_test = test[["text"]].copy()
X_val = val[["text"]].copy()
Y_train = train[["labels"]].copy()
Y_test = test[["labels"]].copy()
Y_val = val[["labels"]].copy()
X_train.head()

,text
0,<user> thanks for showing up for our appointme...
1,haha . # lol
2,i love waiting <num> min for a cab - such shor...
3,22 super funny quotes # funnyquotes # funnysa...
4,goog morning # sorrynotsorry # morning


## Task 3: Pre-process Data

In [6]:
def preprocess(data):
  data = data.astype(str)
  data = data.apply(lambda x: x.lower())
  data = data.apply((lambda x: re.sub('[^a-zA-Z0-9\s]','',x)))

  for idx,row in enumerate(data):
      row = row.replace('rt',' ')
      return data

In [7]:
X_train["text"] = preprocess(X_train["text"])
X_val["text"] = preprocess(X_val["text"])
X_test["text"] = preprocess(X_test["text"])
X_train.head()

,text
0,user thanks for showing up for our appointment...
1,haha lol
2,i love waiting num min for a cab such shortag...
3,22 super funny quotes funnyquotes funnysayi...
4,goog morning sorrynotsorry morning


## Task 4: Choose the Model and its Preprocessor

In [8]:
keras.backend.clear_session()

encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1"
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

## Task 5: Create the Classification model

In [9]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(preprocessor_url, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Dense(2, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [10]:
classifier_model = build_classifier_model()

2025-08-05 03:57:41.590087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-08-05 03:57:41.591879: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-08-05 03:57:41.591946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (educative): /proc/driver/nvidia/version does not exist
2025-08-05 03:57:41.592419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Task 6: Initialize the Parameters

In [11]:
epochs =  4
batch_size = 64
steps_per_epoch = math.floor(X_train.shape[0]/batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
init_lr = 3e-5

In [12]:
loss = tf.keras.losses.BinaryCrossentropy()

metrics = tf.metrics.BinaryAccuracy()

optimizer = optimization.create_optimizer(init_lr=init_lr,
            num_train_steps=num_train_steps, 
            num_warmup_steps=num_warmup_steps,
            optimizer_type='adamw')

## Task 7: Load Trained Model

In [13]:
json_file = open('usercode/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

classifier_model = model_from_json(loaded_model_json, custom_objects={'KerasLayer':hub.KerasLayer})

classifier_model.load_weights("usercode/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [14]:
classifier_model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

## Task 8: Train the Model

In [15]:
classifier_model.compile(optimizer=optimizer,
                      loss=loss,
                      metrics=metrics)

In [16]:
history = classifier_model.fit(X_train, 
pd.get_dummies(Y_train["labels"]),
     validation_data=(X_val, pd.get_dummies(Y_val["labels"])),
     epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/4
310/310 [==============================] - 363s 1s/step - loss: 0.4801 - binary_accuracy: 0.7738 - val_loss: 0.4875 - val_binary_accuracy: 0.7668
Epoch 2/4
310/310 [==============================] - 359s 1s/step - loss: 0.4454 - binary_accuracy: 0.7961 - val_loss: 0.4889 - val_binary_accuracy: 0.7685
Epoch 3/4
310/310 [==============================] - 353s 1s/step - loss: 0.4230 - binary_accuracy: 0.8092 - val_loss: 0.4852 - val_binary_accuracy: 0.7722
Epoch 4/4
310/310 [==============================] - 366s 1s/step - loss: 0.4147 - binary_accuracy: 0.8133 - val_loss: 0.4845 - val_binary_accuracy: 0.7743


## Task 9: Display the Training Curve

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history['binary_accuracy'], label="binary_accuracy")
plt.plot(N, history.history["val_binary_accuracy"], label="val_binary_accuracy")
plt.title("Training Loss and Accuracies (Finetuning BERT for Sarcasm Classification)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

## Task 10: Evaluate the model

In [ ]:
loss, accuracy = classifier_model.evaluate(X_test, pd.get_dummies(Y_test["labels"]))

print('Loss: ', loss)
print('Accuracy: ', accuracy)

In [ ]:
actuals = Y_test["labels"]
Y_predicted = classifier_model.predict(X_test)
predictions= np.argmax(Y_predicted,axis=1)

## Task 11: Create a Confusion Matrix

In [ ]:
cm = confusion_matrix(actuals, predictions)

plt.figure(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')

## Task 12: Generate the Classification Report

In [ ]:
target_names = ['Non-Sarcastic', "Sarcastic"]
print(classification_report(actuals, predictions, target_names=target_names))

# END